### Notebook 10: Hyperparameter tuning

In [1]:
import os
import detectron2
from pathlib import Path
import cv2
import numpy as np
import torch
import json
import pycocotools.mask as mask_util
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog, DatasetMapper
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.data import transforms as T
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.modeling import DatasetMapperTTA
setup_logger()

<Logger detectron2 (DEBUG)>

In [2]:
dataDir=Path('../')
register_coco_instances('sartorius_train',{}, '../sartorius-annotations-coco-format/annotations_train.json', dataDir)
register_coco_instances('sartorius_val',{},'../sartorius-annotations-coco-format/annotations_val.json', dataDir)

In [3]:
score_threshold = [0.204, 0.386, 0.568]
min_mask_area = [75, 150, 75]
def precision_at(threshold, iou):
    matches = iou > threshold
    false_positives = np.sum(matches, axis=0) == 0
    if len(matches.shape)>1:
        false_negatives = np.sum(matches, axis=1) == 0
        true_positives = np.sum(matches, axis=1) == 1
    else:
        false_negatives = 0
        true_positives = 0
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_class = torch.mode(pred['instances'].pred_classes)[0]
    take = pred['instances'].scores >= score_threshold[pred_class]
    pred_masks = pred['instances'].pred_masks[take].cpu().numpy()
    if len(pred_masks)==0:
        return 0.
    else:
        enc_preds = []
        used = np.zeros(pred_masks[0].shape, dtype=int)
        for mask in pred_masks:
            mask = (mask * (1-used)).astype(bool)
            if mask.sum() >= min_mask_area[pred_class]:
                used += mask
                enc_preds.append(mask_util.encode(np.asarray(mask, order='F')) )
        enc_targs = list(map(lambda x:x['segmentation'], targ))
        ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
        prec = []
        for t in np.arange(0.5, 1.0, 0.05):
            tp, fp, fn = precision_at(t, ious)
            p = tp / (tp + fp + fn)
            prec.append(p)
        return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"mAP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=[]))

In [4]:
# No augmentation
cfg = get_cfg()
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("sartorius_train",)
cfg.DATASETS.TEST = ("sartorius_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = 'output_1.1/best_model.pth'
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005
cfg.SOLVER.GAMMA = 0.8
cfg.SOLVER.MAX_ITER = 10000
cfg.SOLVER.STEPS = list(range(2000,10000,1000))
cfg.SOLVER.CHECKPOINT_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.RPN.PRE_NMS_TOPK_TRAIN = 15000
cfg.MODEL.RPN.PRE_NMS_TOPK_TEST = 10000
cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 4000
cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 2000
cfg.MODEL.PIXEL_MEAN = [127.965, 127.965, 127.965]
cfg.MODEL.BACKBONE.FREEZE_AT = 1
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .3
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[24], [40], [80], [128], [256]]
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.33, 0.5, 3.0]]
cfg.MODEL.RPN.IOU_THRESHOLDS = [0.2, 0.7]
cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.RPN.NMS_THRESH = 0.75
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.03
cfg.TEST.DETECTIONS_PER_IMAGE = 700
cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()
os.rename("output", "output_10.1")

[02/08 18:47:55 d2.data.datasets.coco]: Loaded 485 images in COCO format from ../sartorius-annotations-coco-format/annotations_train.json
[02/08 18:47:56 d2.data.datasets.coco]: Loaded 485 images in COCO format from ../sartorius-annotations-coco-format/annotations_train.json
[02/08 18:48:00 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (t

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (9, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (9,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (32, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (32,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (8, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) in th

[02/08 18:48:01 d2.engine.train_loop]: Starting training from iteration 0


/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[02/08 18:48:08 d2.utils.events]:  eta: 0:50:49  iter: 19  total_loss: 3.35  loss_cls: 1.362  loss_box_reg: 0.6466  loss_mask: 0.6934  loss_rpn_cls: 0.3992  loss_rpn_loc: 0.2887  time: 0.3142  data_time: 0.0243  lr: 9.9905e-06  max_mem: 2646M
[02/08 18:48:15 d2.utils.events]:  eta: 0:51:25  iter: 39  total_loss: 3.361  loss_cls: 1.287  loss_box_reg: 0.6772  loss_mask: 0.6877  loss_rpn_cls: 0.358  loss_rpn_loc: 0.3167  time: 0.3292  data_time: 0.0359  lr: 1.998e-05  max_mem: 3235M
[02/08 18:48:23 d2.utils.events]:  eta: 0:51:45  iter: 59  total_loss: 3.111  loss_cls: 1.113  loss_box_reg: 0.7385  loss_mask: 0.6725  loss_rpn_cls: 0.3235  loss_rpn_loc: 0.2469  time: 0.3462  data_time: 0.0640  lr: 2.997e-05  max_mem: 3244M
[02/08 18:48:30 d2.utils.events]:  eta: 0:52:30  iter: 79  total_loss: 2.892  loss_cls: 0.8857  loss_box_reg: 0.8778  loss_mask: 0.6395  loss_rpn_cls: 0.2595  loss_rpn_loc: 0.2472  time: 0.3437  data_time: 0.0124  lr: 3.9961e-05  max_mem: 3244M
[02/08 18:48:37 d2.utils.ev

In [4]:
# Default augmentations + we add more augmentations
class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=[
        T.RandomContrast(0.95,1.05),
        T.RandomBrightness(0.95,1.05),
        T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
        T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
        T.ResizeShortestEdge(short_edge_length=(832, 864, 896, 928, 960, 992, 1024), max_size=9999, sample_style='choice')
        ]))

cfg = get_cfg()
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.INPUT.MIN_SIZE_TEST = 1024
cfg.INPUT.MAX_SIZE_TEST = 3000
cfg.DATASETS.TRAIN = ("sartorius_train",)
cfg.DATASETS.TEST = ("sartorius_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = 'output_1.1/best_model.pth'
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005
cfg.SOLVER.GAMMA = 0.8
cfg.SOLVER.MAX_ITER = 10000
cfg.SOLVER.STEPS = list(range(2000,10000,1000))
cfg.SOLVER.CHECKPOINT_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.RPN.PRE_NMS_TOPK_TRAIN = 15000
cfg.MODEL.RPN.PRE_NMS_TOPK_TEST = 10000
cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 4000
cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 2000
cfg.MODEL.PIXEL_MEAN = [127.965, 127.965, 127.965]
cfg.MODEL.BACKBONE.FREEZE_AT = 1
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .3
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[24], [40], [80], [128], [256]]
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.33, 0.5, 3.0]]
cfg.MODEL.RPN.IOU_THRESHOLDS = [0.2, 0.7]
cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.RPN.NMS_THRESH = 0.75
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.03
cfg.TEST.DETECTIONS_PER_IMAGE = 700
cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()
os.rename("output", "output_10.2")

[02/08 22:49:48 d2.data.datasets.coco]: Loaded 485 images in COCO format from ../sartorius-annotations-coco-format/annotations_train.json
[02/08 22:49:49 d2.data.datasets.coco]: Loaded 485 images in COCO format from ../sartorius-annotations-coco-format/annotations_train.json
[02/08 22:49:52 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (t

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (9, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (9,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (32, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (32,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (8, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) in th

[02/08 22:49:53 d2.engine.train_loop]: Starting training from iteration 0


/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[02/08 22:50:11 d2.utils.events]:  eta: 2:09:43  iter: 19  total_loss: 3.498  loss_cls: 1.525  loss_box_reg: 0.5863  loss_mask: 0.6938  loss_rpn_cls: 0.3226  loss_rpn_loc: 0.3108  time: 0.8616  data_time: 0.1723  lr: 9.9905e-06  max_mem: 7132M
[02/08 22:50:29 d2.utils.events]:  eta: 1:55:51  iter: 39  total_loss: 3.123  loss_cls: 1.39  loss_box_reg: 0.4175  loss_mask: 0.6885  loss_rpn_cls: 0.3299  loss_rpn_loc: 0.2339  time: 0.8867  data_time: 0.2375  lr: 1.998e-05  max_mem: 9270M
[02/08 22:50:46 d2.utils.events]:  eta: 1:53:59  iter: 59  total_loss: 2.944  loss_cls: 1.166  loss_box_reg: 0.521  loss_mask: 0.6711  loss_rpn_cls: 0.3053  loss_rpn_loc: 0.2501  time: 0.8589  data_time: 0.1495  lr: 2.997e-05  max_mem: 9270M
[02/08 22:51:03 d2.utils.events]:  eta: 1:55:23  iter: 79  total_loss: 2.723  loss_cls: 0.9494  loss_box_reg: 0.6657  loss_mask: 0.6623  loss_rpn_cls: 0.2553  loss_rpn_loc: 0.2281  time: 0.8608  data_time: 0.1652  lr: 3.9961e-05  max_mem: 9270M
[02/08 22:51:20 d2.utils.ev

In [3]:
def print_metrics(output):
    with open(f"output_{output}/metrics.json",'r') as f:
        metrics = [json.loads(line) for line in f]
    print("mAP :", np.mean([metrics[i]["mAP IoU"] for i in range(len(metrics)) if 'mAP IoU' in metrics[i]][-10:]))
    print("False negatives :", np.mean([metrics[i]["mask_rcnn/false_negative"] for i in range(len(metrics)) if 'mask_rcnn/false_negative' in metrics[i]][-100:]))
    print("False positives :", np.mean([metrics[i]["mask_rcnn/false_positive"] for i in range(len(metrics)) if 'mask_rcnn/false_positive' in metrics[i]][-100:]))
i=1
while True:
    try:
        print("Experiment ",i)
        print_metrics("10."+str(i))
        i+=1
    except:
        break

Experiment  1
mAP : 0.2103876271902451
False negatives : 0.10366114738905222
False positives : 0.17422062519708162
Experiment  2
mAP : 0.3002627002133756
False negatives : 0.10839094834021118
False positives : 0.1582021753237467
Experiment  3
